In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To install dependencies

Note: After installation, please click "RESTART RUNTIME" to restart the runtime.

In [ ]:
!pip install ultralytics
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 55.5 MB/s eta 0:00:00


Milestone 1 (Player detection using the YOLOv8 models trained on COCO dataset)

In [ ]:
import sys
root_path = '/content/drive/MyDrive/basketball-broadcaster'
sys.path.append(root_path)

import os
os.chdir(root_path)

import cv2
from yolov8 import YOLOv8
import time
import av
import numpy as np
from PIL import Image


You can 3 kinds of YOLOv8 models

In [ ]:
player_detector = YOLOv8('yolov8m.pt') # 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt' or 'yolov8x.pt'

Inference on a video file: the result video is outputtted in the drive root folder. For the example video below, the file "test1_clip_out.mp4" is outputted.

In [ ]:
input_ = av.open("videos/test1_clip.mp4")
output = av.open("output.mp4", "w")

# Make an output video stream using the input video stream as a template
in_video_stream = input_.streams.video[0]
out_video_stream = output.add_stream(codec_name="h264",
                                     width=in_video_stream.width,
                                     height=in_video_stream.height,
                                     rate=int(in_video_stream.average_rate),
                                     bit_rate = in_video_stream.bit_rate,
                                     pix_fmt=in_video_stream.pix_fmt)

# Make an output audio stream using the input audio stream as a template
in_audio_stream = input_.streams.audio[0]
out_audio_stream = output.add_stream(template=in_audio_stream)


for packet in input_.demux():
    # We need to skip the "flushing" packets that `demux` generates.
    if packet.dts is None:
        continue

    if packet.stream == in_video_stream:
        for frame in packet.decode():
            # To get an input frame
            image = frame.to_image()
            image = np.asarray(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # To process the input frame
            boxes, scores, class_ids = player_detector.detect(image)
            image = player_detector.draw_detection(image, boxes, scores, class_ids)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)

            output_frame = av.VideoFrame.from_image(image)
            output_frame.pts = None
            for output_packet in out_video_stream.encode(output_frame):
                # Write the packets to the container
                output_packet.stream = out_video_stream
                output.mux(output_packet)
    elif packet.stream == in_audio_stream:
        packet.stream = out_audio_stream
        output.mux(packet)

input_.close()
output.close()


0: 384x640 27 persons, 6 chairs, 1 potted plant, 113.4ms
Speed: 18.4ms preprocess, 113.4ms inference, 45.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 27 persons, 6 chairs, 1 potted plant, 26.1ms
Speed: 3.6ms preprocess, 26.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 persons, 1 tennis racket, 6 chairs, 1 potted plant, 26.1ms
Speed: 3.1ms preprocess, 26.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 persons, 1 tennis racket, 6 chairs, 1 potted plant, 26.1ms
Speed: 4.3ms preprocess, 26.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 26 persons, 3 tennis rackets, 6 chairs, 1 potted plant, 26.0ms
Speed: 2.6ms preprocess, 26.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 26 persons, 2 tennis rackets, 5 chairs, 1 potted plant, 26.3ms
Speed: 2.9ms preprocess, 26.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0